#Python code to extract ship and voyage info from csv

**Data background:**

Data is ship authority files, mostly based on entries from Farrington's *Catalogue of East India Company Ships' Journals and Logs*, which was keyed at some point in the early 2000s and later imported to the IAMS system. The entries are formatted pretty consistently in the catalogue, which was generally replicated in the keying, although subsequent cataloguer amendments have added irregularities.

I'm using a subset of columns from the full files: ID, SHIP NAME, DATE RANGE, HISTORY.

History fields contain text, often giving info like contract type, size, builder, owner, followed by details of voyages, which can be multiple. Voyages are numbered, and typically record the season of voyage with destination, followed by captain (if known), followed by stages.

E.g.


'Chartered ship, 32/35 crew, 450 tons. Principal Managing Owner: William Bawtree. Voyages: (1) 1818/9 Bengal. Capt Lucas Percival. Downs 27 May 1819 - 30 Sep Bengal - 29 Dec Narsipur - 3 Jan 1820 Madras - 22 Mar St Helena - 13 May East India Dock. (2) 1822/3 Bengal. Capt Lucas Percival. Downs 25 May 1829 - 21 Sep Hugli - 14 Oct Calcutta - 29 Jan 1824 Saugor - 2 Apr St Helena - 17 Jun East India Dock.'...

Most entries were entered with individual start and end dates expressed as '-9999', and date ranges as 'Undetermined'. I'm not looking to fix those here for the moment, as there's a separate project working to update the authority files. Any improvements introduced here can later be linked by using the unique authority IDs. I'm focused here on extracting voyage data.

**What the script does:**

Iterate over csv, create dictionary for each ship recording basic info (ship name, id, info), and extracting voyage data from free text 'history' column, with voyages recorded as list of lists containing tuples which record date and location. Individual ship dictionaries then added to a main dictionary, with ID as key.

This one has basic error handling to record problem records and then continue running. (My first time using try / except / else and it probably shows!)

With a sample dataset of 97 records, 40 had to be passed. The full  dataset consists of c1500 entries.

![book_cover](book_cover_voyage_text.png "Book Cover of Farrington's")

In [13]:
# IMPORT STATEMENTS
import json
import re
import pandas as pd
import requests

In [2]:
ships_df = pd.read_csv("../data/raw/ship_name_history.csv", index_col="RecordID")

In [ ]:
ships_df.head(10)

In [ ]:
place_date_regex = re.compile(r"(?P<Location>[a-zA-Z\s']*\b)? ?(?P<Date>(\d{1,2}\s)?\w{3}(\s\d{4})?)?")
date_place_regex = re.compile(r"(?P<Date>(\d{1,2}\s)?\w{3}(\s\d{4})?)? ?(?P<Location>\b[a-zA-Z\s'-]*\b)")
duration_dest_regex = re.compile(r"(?P<Duration>\b[\d/-]*\b) ?(?P<Destination>[\s\w,&--'\(\)]*)?.?$")

ship_voyages = []
voyage_part_parse_failures = []
dur_date_failures = []
date_place_failures = []
place_date_failures = []

for ship_id, row in ships_df.iterrows():
    ship_info = {
        "name": row["CorporateName"],
        "dates": row["DateRange"],
        "info": "",
        "voyages": [],
        "raw_history": row["History"]
    }

    voyages = []
    if type(row["History"]) != str:
        ship_info["info"] = "No history recorded"
        ship_voyages.append({ship_id: ship_info})
        continue

    if "Voyages: " in row["History"]:
        info, voyage_string = row["History"].split("Voyages: ")
        ship_info["info"] = info.strip()
    else:  # No voyage information
        ship_info["info"] = row["History"]
        ship_voyages.append({ship_id: ship_info})
        continue
    
    
    raw_voyages = [x.strip() for x in re.split(r"\(\d{1,2}\) ", voyage_string) if x]  # First item in list is empty string due to split around first bracketed voyage number (1) 
    for rv in raw_voyages:
        voyage = {
            "duration": "",
            "start_date": "",
            "end_date": "",
            "destination": "",
            "captain": "",
            "route": [],
            "parse_failure": False
        }

        voyage_parts = [x.strip() for x in rv.split(".") if x]           
        try:
            if ("Capt" in rv or "Master" in rv) and "-" in rv:
                duration_dest, captain, route_str = voyage_parts[:3]
            elif ("Capt" in rv or "Master" in rv) and "-" not in rv:
                duration_dest, capt = voyage_parts[:2]
            elif "-" in rv:
                duration_dest, route_str = voyage_parts[:2]
            elif len(voyage_parts) == 2 and "-" not in rv:
                duration_dest, route_str = voyage_parts
            elif "-" not in rv:
                duration_dest = rv
        except ValueError:
            voyage_part_parse_failures.append((ship_id, rv))
            voyage["route"].append(rv)
            voyage["parse_failure"] = True
            voyages.append(voyage)
            continue

        try:
            dd_match = duration_dest_regex.match(duration_dest)
            duration, destination = dd_match.group("Duration"), dd_match.group("Destination")
        except AttributeError as e:
            dur_date_failures.append((ship_id, duration_dest))
            voyage["route"].append(rv)
            voyage["parse_failure"] = True
            voyages.append(voyage)
            continue

        voyage["captain"] = captain
        voyage["duration"] = duration
        voyage["destination"] = destination

        raw_stops = route_str.split(" - ")
        stops = []

        try:
            start = place_date_regex.search(raw_stops[0])
            start_location, start_date = start.group("Location"), start.group("Date")
            if start_location:
                start_location = start_location.strip()
        except AttributeError:
            stops.append({"Unparsed stop": stop})
            voyage["parse_failure"] = True
            place_date_failures.append((ship_id, raw_stops[0]))
            
        voyage["start_date"] = start_date
        
        stops.append({start_date: start_location})

        for stop in raw_stops[1:]:
            dp_match = date_place_regex.match(stop)
            if dp_match:
                loc, date = dp_match.group("Location").strip(), dp_match.group("Date")
                stops.append({date: loc})
            elif not date and re.search(r"\d", stop):  # Check if it's actually place/date format
                pd_match = place_date_regex.match(stop)
                pd_loc, pd_date = pd_match.group("Location").strip(), pd_match.group("Date")
                if pd_date:
                    loc, date = pd_loc, pd_date
                    stops.append({date: loc})
                else:
                    date_place_failures.append((ship_id, stop))
                    stops.append({"unable_to_date": stop})
                    voyage["parse_failure"] = True                       
            else:
                date_place_failures.append((ship_id, stop))
                stops.append({"unable_to_date": stop})
                voyage["parse_failure"] = True    

        if len(voyage_parts) > 3:
            [stops.append({"Additional voyage": p}) for p in voyage_parts[3:]]
            
        voyage["route"] = stops
        voyage["end_date"] = [x for x in stops[-1].keys()][0]

        voyages.append(voyage)

    ship_info["voyages"] = voyages

    ship_voyages.append({ship_id: ship_info})

In [ ]:
len(ship_voyages), len(voyage_part_parse_failures), len(dur_date_failures), len(date_place_failures), len(place_date_failures)

In [ ]:
date_place_failures

In [ ]:
place_date_regex = re.compile(r"(?P<Location>[a-zA-Z\s']*\b)? ?(?P<Date>(\d{1,2}\s)?\w{3}(\s\d{4})?)?")
date_place_regex = re.compile(r"(?P<Date>(\d{1,2}\s)?\w{3}(\s\d{4})?)? ?(?P<Location>\b[a-zA-Z\s'-]*\b)")
duration_dest_regex = re.compile(r"(?P<Duration>\b[\d/-]*\b) ?(?P<Destination>[\s\w,&--'\(\)]*)?.?$")

In [ ]:
ship_voyages[1]

Types of `History` string:
 - Ship info and voyage info. Start with ship info then `Voyages:` and voyage info
 - Only voyage info, string starts with `Voyages:` and has voyage info only

The voyages part is typically individual voyages in short text separated by voyage numbers in round brackets e.g. (1)
Types of individual voyage string:
- Years duration and a destination, then a captain, then text describing the stops on the voyage.
Types of voyage string inconsistency:
- No captain, just duration/destination then stops
- No stops, just duration/destination then captain
- No destination, just duration then captain/stops
- No captain or stops
- Poorly formatted: misplaced `.`, `-`
- Journey variation: wrecked, didn't return

At current all 'voyage_part_parse_failures' are due to missing '.' between parts of the voyage.

The duration/destination can also vary:
- Unhandled characters in the duration/destination text

### Most basic workflow with sanitised data

No conditionals to check to make logic more flexible

Can also use this to find just the records that are parsable with the basic logic

In [38]:
def teaching_parse(ship_id, row):
    date_place_regex = re.compile(r"(?P<Date>\d{1,2} \w{3}( \d{4})?) (?P<Location>\b[\w\s]*\b)")
    place_date_regex = re.compile(r"(?P<Location>\b[\w\s]*\b) (?P<Date>\d{1,2} \w{3} \d{4})")
    
    ship_info = {
        "name": row["CorporateName"],
        "dates": row["DateRange"],
        "info": "",
        "voyages": [],
        "raw_history": row["History"]
    }

    voyages = []
    info, voyage_string = row["History"].split("Voyages: ")
    ship_info["info"] = info.strip()
    
    raw_voyages = re.split(r"\(\d{1,2}\) ", voyage_string)[1:]  # First item in list is empty string due to split around first bracketed voyage number (1) 
    for i, rv in enumerate(raw_voyages):
        voyage = {
            "voyage_number": i + 1,
            "duration": "",
            "destination": "",
            "captain": "",
            "route": []
        }

        duration_dest, captain, route_str = rv.split(". ")[:3]
        duration, destination = duration_dest.split(" ")[:2]

        voyage["captain"] = captain
        voyage["destination"] = destination
        voyage["duration"] = duration

        raw_stops = route_str.split(" - ")
        stops = []
        
        start = place_date_regex.search(raw_stops[0])
        start_location, start_date = start.group("Location"), start.group("Date")
        
        stops.append({start_date: start_location})

        for stop in raw_stops[1:]:
            match = date_place_regex.search(stop)
            loc, date = match.group("Location"), match.group("Date")
            stops.append({date: loc})

        voyage["route"] = stops

        voyages.append(voyage)

    ship_info["voyages"] = voyages
    
    return ship_info

In [28]:
clean_voyages = []

for ship_id, row in ships_df.iterrows():
    try:
        simple_parse(ship_id, row)
        clean_voyages.append(ship_id)
    except:
        continue

In [29]:
len(clean_voyages)

728

In [ ]:
ships_df.loc[pd.Index(clean_voyages)].query("DateRange != 'Unspecified'").iloc[:20].to_csv("../data/raw/ships_sample.csv", encoding="utf8")

In [7]:
sample_df = pd.read_csv("../data/raw/ships_sample.csv", index_col=0, encoding="utf8")

In [39]:
ship_voyages = []

for ship_id, row in sample_df.iterrows():
    ship_info = teaching_parse(ship_id, row)
    ship_voyages.append({ship_id: ship_info})

In [9]:
idx = 19
(sample_df.iloc[idx],sample_df.iloc[idx]["History"])

(CorporateName                                            Godolphin
 History          Rated at 499 tons, 99 crew, 30 guns. Voyages: ...
 DateRange                                                1734-1746
 Name: 045-001115063, dtype: object,
 'Rated at 499 tons, 99 crew, 30 guns. Voyages: (1) 1734/5 Madras and Bengal. Capt Francis Steward. Downs 20 Jan 1735 - 7 Apr Cape - 16 Jun Madras - 6 Jul Rogues River - 13 Oct Ingeli - 17 Dec Madras - 9 Mar 1736 Cape - 27 Mar St Helena - 6 Jul Downs. (2) 1737/8 Madras and China. Capt Francis Steward. Downs 12 Oct 1737 - 7 Jan 1738 Cape - 5 May Madras - 8 Jul Malacca - 14 Aug Whampoa - 4 Nov Second Bar - 24 Nov Malacca - 22 Dec Madras - 2 May 1739 St Helena - 13 Jul Downs. (3) 1740/1 Bombay and China. Capt Francis Steward. Downs 21 Jul 1740 - 6 Dec Cape - 23 Mar 1741 Cochin - 26 Mar Calicut - 28 Mar Tellicherry - 19 Jun Malacca - 17 Jul Whampoa - 6 Dec Second Bar - 12 Apr 1742 Cape - 24 May St Helena - 6 Sep Downs. (4) 1743/4 Benkulen and Bengal. Ca

In [10]:
[print(s) for s in sample_df.iloc[idx]["History"].split("Voyages")[1].split(" - ")]

: (1) 1734/5 Madras and Bengal. Capt Francis Steward. Downs 20 Jan 1735
7 Apr Cape
16 Jun Madras
6 Jul Rogues River
13 Oct Ingeli
17 Dec Madras
9 Mar 1736 Cape
27 Mar St Helena
6 Jul Downs. (2) 1737/8 Madras and China. Capt Francis Steward. Downs 12 Oct 1737
7 Jan 1738 Cape
5 May Madras
8 Jul Malacca
14 Aug Whampoa
4 Nov Second Bar
24 Nov Malacca
22 Dec Madras
2 May 1739 St Helena
13 Jul Downs. (3) 1740/1 Bombay and China. Capt Francis Steward. Downs 21 Jul 1740
6 Dec Cape
23 Mar 1741 Cochin
26 Mar Calicut
28 Mar Tellicherry
19 Jun Malacca
17 Jul Whampoa
6 Dec Second Bar
12 Apr 1742 Cape
24 May St Helena
6 Sep Downs. (4) 1743/4 Benkulen and Bengal. Capt John Stevens. Downs 31 Mar 1744
29 Aug Benkulen
17 Dec Ryapore
14 Feb 1745 Ingeli
17 Jun St Helena
17 Sep Galway
20 Dec Downs.


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [40]:
def to_json(voyages):
    voyage_dict = {}
    for v in voyages:
        voyage_dict |= v

    with open("../data/processed/ships.json", "w") as f:
        json.dump(voyage_dict, f, indent="\t")

In [41]:
to_json(ship_voyages)

In [45]:
def from_json(fp):
    with open(fp, "r") as f:
        voyage_dict = json.load(f)

    return [{k:v} for k,v in voyage_dict.items()]

In [ ]:
def test_output():
    

In [46]:
from_json("../data/processed/ships.json")

[{'045-001114649': {'name': 'Boscawen',
   'dates': '1748-1765',
   'info': 'Rated at 499 tons, 26 guns, 99 crew. Principal Managing Owner: 4 Richard Crabb.',
   'voyages': [{'voyage_number': 1,
     'duration': '1748/9',
     'destination': 'Bombay',
     'captain': 'Capt Benjamin Braund',
     'route': [{'26 Mar 1749': 'Downs'},
      {'5 Jul': 'Johanna'},
      {'2 Aug': 'Bombay'},
      {'22 Sep': 'Surat'},
      {'17 Nov': 'Bandar Abbas'},
      {'23 Dec': 'Bombay'},
      {'11 Feb 1750': 'Mangalore'},
      {'17 Feb': 'Tellicherry'},
      {'19 Mar': 'Socotra'},
      {'29 Mar': 'Mokha'},
      {'27 Aug': 'Bombay'},
      {'16 Jan 1751': 'Cape'},
      {'17 Feb': 'St Helena'},
      {'4 Jun': 'Gravesend'}]},
    {'voyage_number': 2,
     'duration': '1752/3',
     'destination': 'Madras',
     'captain': 'Capt Benjamin Braund',
     'route': [{'27 Dec 1752': 'Downs'},
      {'15 Mar 1753': 'Cape'},
      {'24 Jun': 'Madras'},
      {'9 Sep': 'Whampoa'},
      {'26 Dec': 'Second B

In [11]:
ship_voyages[idx]

{'045-001115063': {'name': 'Godolphin',
  'dates': '1734-1746',
  'info': 'Rated at 499 tons, 99 crew, 30 guns.',
  'voyages': [{'duration': '1734/5',
    'destination': 'Madras',
    'captain': 'Capt Francis Steward',
    'route': [{'20 Jan 1735': 'Downs'},
     {'7 Apr': 'Cape'},
     {'16 Jun': 'Madras'},
     {'6 Jul': 'Rogues River'},
     {'13 Oct': 'Ingeli'},
     {'17 Dec': 'Madras'},
     {'9 Mar 1736': 'Cape'},
     {'27 Mar': 'St Helena'},
     {'6 Jul': 'Downs'}]},
   {'duration': '1737/8',
    'destination': 'Madras',
    'captain': 'Capt Francis Steward',
    'route': [{'12 Oct 1737': 'Downs'},
     {'7 Jan 1738': 'Cape'},
     {'5 May': 'Madras'},
     {'8 Jul': 'Malacca'},
     {'14 Aug': 'Whampoa'},
     {'4 Nov': 'Second Bar'},
     {'24 Nov': 'Malacca'},
     {'22 Dec': 'Madras'},
     {'2 May 1739': 'St Helena'},
     {'13 Jul': 'Downs'}]},
   {'duration': '1740/1',
    'destination': 'Bombay',
    'captain': 'Capt Francis Steward',
    'route': [{'21 Jul 1740': 'Do

### Alex's questions

#Next steps proposals / questions

Run over the entire dataset. Take the IDs of problematic records and remove from dataset, then re-run on that to get an initial output and a set of problematic records for further examination.

Examine initial output:

*   Close reading to flag any errors
*   Write code to flag voyage lists with len < 3
*   Write code to flag voyage steps which look like this ('23 Sep', '1685.')
*   Subsequent data cleaning / refining code


Problem entries:

*   Close reading to see what the problems might be and how to approach
*   Possibly use a more granular approach. Split by '-' then look more closely?
*   Run against gazatteer or use NER to identify place names and find closest dates?

Other questions:

*  Are nested dictionaries the best way of structuring this output? Would JSON be better?

*  Ultimately I would like to take the voyage steps, geolocate the places and tidy the dates so that they can be queried (which ships pass place X within timespan Y) and plotted, although there's a lot of data cleaning to do before then.


### Can I just use an LLM?

Yes, and they can produce good results. The reason I haven't suggested them at the start is because this tutorial is about how to write Python, not how to prompt an LLM. Using LLMs for work tasks also raises a range of ethical considerations. Read the BL's AI Principles and explore a framework like the Library of Congress' Labs [AI Planning Framework](https://libraryofcongress.github.io/labs-ai-framework/) to help you understand the benefits and risks of carrying out this work at scale.

Let's explore using an LLM as extra credit now you've done the bulk of your learning. LLMs are quite good at extracting structured data from unstructured text [references]. At the time of writing my impression is that Anthropic have the best governance processes, so open https://claude.ai and sign up for an account (~1 min). Then you can start putting in sections of the text and trying to get Claude to extract the data in a format similar to that above. Finding the right prompt is important, and is one of the skills needed to fruitfully interact with language models. Experiment yourself or make use of the one below, which I've adapted from [Matt Miller](https://thisismattmiller.com/post/using-gpt-on-library-collections/).

--- 

You are a helpful assistant that is extracting data from a ship voyage information. You only answer using the text given to you. You do not make-up additional information, the answer has to be contained in the text provided to you. Each voyage is a string of text. 
You will structure your answer in valid JSON, extract the date in the format yyyy-mm-dd and the location the ship visited using the JSON keys dateVisited and location.

If the following text contains multiple voyages, extract each one into an array of 
valid JSON dictionaries. Each dictionary represents one of the entries:

Downs 27 May 1819 - 30 Sep Bengal - 29 Dec Narsipur - 3 Jan 1820 Madras - 22 Mar St Helena - 13 May East India Dock

---